<figure>
    <img src="../images/tudelft_logo.png" alt="image" width="250" align="right"/>
</figure>

# 2c: Generation of the tide

This notebook is the third of four notebooks accompanying Chapter 3 of the [Coastal Dynamics Open Textbook](https://books.open.tudelft.nl/home/catalog/book/202). It specifically addresses the generation of the tide through the interplay between gravitational attraction forces in the Earth-Moon and Earth-Sun systems, respectively (see Chapter 3.7 of the book and the corresponding slides). The notebook explores these concepts in a more interactive way, with some exercises, visualizations, and code. There are also ten questions included in this notebook. We recommend that you read the text carefully, scroll through the code to really understand the computations and study the figures. We will not repeat the whole theory, however, so make sure you have followed the lectures for this week and read the relevant pages in Chapter 3 of the book. 

We will focus on the following concepts:
1. Differential pull (Part 1, 6 questions)
2. Equilibrium response and spring-neap tide (Part 2, 4 questions)

## Import packages
Run the cell below to load the libraries and questions used in this notebook.

In [ ]:
from pathlib import Path

import numpy as np
import panel as pn

import coastal_dynamics as cd

pn.extension()

In [ ]:
import sys

sys.path.append('../')

from modules import mod_2c

In [ ]:
questions = cd.read_questions(Path("../hashed_questions/2c_generation_tide_hashed.json"))

question_industry = cd.QuestionIndustry(questions)

cd.UseAnswersApp("2c").serve()

## Part 1: Differential pull

First, we will have a look at the generation of the tide. To start, we are going to look at the gravitational forces between the Earth, Moon, and Sun. As you have learned, the tide is generated by the so-called *differential pull*, meaning the difference in gravitational pull on water masses at different locations on the Earth as compared to the gravitational pull for the centre of the earth. 

### Gravitational pull between two bodies

Celestial bodies (e.g. the Earth and the Moon) orbit around a common centre of mass. This implies that a force is present that accelerates the bodies towards the common centre of mass (if there were no force, the bodies would either be at rest or move at constant speed in a straight line - Newton's first law). Since the orbits are the same for every location on a celestial body, the acceleration is also the same for every location. The force responsible for the inward acceleration is the gravitational force between celestial bodies. 

The Earth is pulled towards the Moon as well as towards the Sun. We can compute the magnitude of this pulling force by using Newton's law of universal gravitation. This law states that every particle attracts every other particle in the universe with a force $F$ that is proportional to the product of their masses and inversely proportional to the square of the *distance between their centres*:
$$
F = G \frac{m_1 m_2}{r^2}
$$

Here, $G$ is the universal gravitational constant. The distance $r$ is the distance between the centres of the celestial bodies (the average distance between them). The formula thus provides the gravitational pull between two celestial bodies for the bodies as a whole. It is implemented as a function in the first cell below.

### Gravitational pull of Sun and Moon on Earth
For the gravitational pull on Earth, we need to take the forces into account that the Moon and the Sun exert on the Earth. For this notebook, we will compute the forces that are working on 1 kg of Earth mass, to keep it simple. Hence, the mass we will fill in for the Earth will be 1 kg. Also, we will express the forces in terms of the Earths own gravitational acceleration ($g$), which makes it easier to compare the magnitudes with each other. 

Take the following steps:
1. Check the computation procedure followed in the first and second cell below. In the first cell, the universal gravitational law is defined. Using this function as defined in the first cell, the second cell computes the gravitational pull of the Moon and Sun on the Earth per kg of mass on Earth.
2. Run the third cell below. It provides you with two questions to reflect on the computational results for the gravitational pull of the Moon and Sun on Earth. 

In [ ]:
### Functions
def grav_pull(m1, m2, d):
    """
    This function calculates the gravitational pull between two celestial bodies using Newton's law of universal gravitation.

    Parameters:
    m1 (float): mass of the first body in kg.
    m2 (float): mass of the second body in kg.
    d (float): distance between the two bodies in meters.

    Returns:
    g_pull (float): the gravitational pull between the two bodies in g

    """
    G = 6.67408e-11  # universal gravitational constant
    g = 9.81  # gravitational acceleration on the Earth
    g_pull = G * (m1 * m2) / d**2  # gravitational pull between two bodies

    # we divide the GP by g, so we get the gravitational pull in terms of g
    return g_pull / g

In [ ]:
## Run this cell to compute the gravitational pull of the Sun and Moon on 1 kg of mass on Earth
## Properties of Earth, Moon, and Sun
m_earth = 1  # mass of the earth in kg (we use 1 to compute the gravitational pull per kg of earth mass)
m_sun = 1.99e30  # mass of the sun in kg
m_moon = 7.35e22  # mass of the moon in kg
d_sun = 1.5e11  # distance between the centres of the sun and earth in meters
d_moon = 3.84e8  # distance between the centres of the moon and earth in meters
r = 6.37e6  # radius of the Earth in meters

# Now we compute the gravitational pull and print them
gp_sun = grav_pull(m_earth, m_sun, d_sun)
gp_moon = grav_pull(m_earth, m_moon, d_moon)

# Print out results
print("Gravitational pull from the Sun on 1 kg of mass on Earth using the distance between the centres of Earth and Sun:  %.2E g" % gp_sun)
print("Gravitational pull from the Moon on 1 kg of mass on Earth using the distance between the centres of Earth and Moon: %.2E g" % gp_moon)

In [ ]:
## Run this cell to get question(s)

q = [
    "Q2c_grav_pull_sun",
    "Q2c_grav_pull_moon"
]

question_industry.serve(q)

### Variation of the gravitational pull on Earth

Above we have computed the gravitational pull using the average distance between the Earth and either the Sun or the Moon. We found that the gravitational pull of the Sun is about two orders of magnitude larger than that of the Moon, and both are much smaller than $g$. Yet, we know that these forces are somehow responsible for tidal ranges of multiple meters, and that the moon is actually responsible for about 70% of these tides. How come? 

As we can see in the formula, the gravitational pull is strongly dependent on the distance between the two celestial bodies. Since the Earth is a sphere, different locations on the Earth's surface will experience subtle differences in the gravitational pull, as the distance to the Sun/Moon varies slightly over the Earth's surface. 

Let's use the above function to compute the gravitational pull from the Sun at 8 different locations around the Earth's surface (A-H, see the figure below). Also the already computed value for the centre of the Earth is taken along (as point X). By changing the code, you can also make the computation for the Moon.

![earth_positions](../images/2_wind_waves_tides/02_earth_sun_gp.png)

In [ ]:
# First we compute the distance between the centre of the Sun and each position on Earth
distance = d_sun  # switch to d_moon to change the computation to Moon
distances = {
    "X": distance,
    "A": np.sqrt(r**2 + distance**2),  # Same as point E
    "B": np.sqrt(
        (r * np.sin(45 * np.pi / 180)) ** 2
        + (distance - r * np.cos(45 * np.pi / 180)) ** 2
    ),  # Same as point D
    "C": distance - r,
    "D": np.sqrt(
        (r * np.sin(45 * np.pi / 180)) ** 2
        + (distance - r * np.cos(45 * np.pi / 180)) ** 2
    ),  # Same as point B
    "E": np.sqrt(r**2 + distance**2),  # Same as point A
    "F": np.sqrt(
        (r * np.sin(45 * np.pi / 180)) ** 2
        + (r * np.cos(45 * np.pi / 180) + distance) ** 2
    ),  # Same as point H
    "G": distance + r,
    "H": np.sqrt(
        (r * np.sin(45 * np.pi / 180)) ** 2
        + (r * np.cos(45 * np.pi / 180) + distance) ** 2
    ),  # Same as point F
}

# Now compute the gravitational pull for each point.
gp = {}
for pos in distances:
    gp[pos] = grav_pull(m_earth, m_sun, distances[pos])
    print("Gravitational pull at position %s: %.5E g" % (pos, gp[pos]))

### Visualization of differential pull
 
As you can see, the differences are very subtle (in the order of $10^{-7} g$). We know that the gravitational pull of the Moon and the Sun are responsible for the Earth's orbital motion in the Earth-Moon and Earth-Sun systems, respectively. This orbital motion requires a gravitional pull that is the same for every point on Earth and equal to the pull computed for the centre of the Earth. In other words, since the Earth moves as one body, it does not respond to the variations in gravitational pull. The water masses on the surface of the solid Earth, however, can move around and are not as fixed and rigid as the solid land masses. So, while the solid masses of the Earth remain virtually unchanged, its water masses are moved around by these subtle differences in gravitational pull.

So, let's visualize this differential pull for the Sun. The orbital motion of the Earth in the Earth-Sun system is caused by the gravitational pull of the Sun for the Earth as a whole, so computed using the distance between the centres of Earth and Sun. This will be our *reference pull*. For the other locations around the Earth, we will compute the difference from the reference pull, by subtracting the reference pull from the gravitational pull at each location. If we do this with the actual forces we computed above, we won't see the difference since they are very small. Therefore, we will visualize the forces out of proportion, i.e., we exaggerate the difference. Next to the difference in magnitude, there are also slight differences in the direction of the forces at different positions, as each force is directed towards the centre of the Sun/Moon. We will also visualize these out of proportion, so we can actually see the difference. Run the code cell below to get a visualization of the forces we talked about above.

The red arrows indicate the gravitational pull at each individual location around Earth's surface. The black lined, transparent arrows represent the reference pull and are all equal to the black arrow at the centre (X). If we subtract the black arrows from the red arrows at each location, we end up with the green arrows in the second plot, which we call the differential pull. Do the green arrows already give you an indication of the shape of the oceanic waters due to the tide?

In [ ]:
mod_2c.plot_grav_pull()

### Near-side differential pull and contribution of Sun and Moon

Now, let's compute the actual differential pull at location C, for both the Earth-Sun and the Earth-Moon system. 

Run the below two cells for the computation and a couple of reflective questions about the computation procedure and results.

In [ ]:
## Compute the differential pull for both the Earth-Sun and the Earth-Moon system. 
## Location C
dp_C_sun = grav_pull(m_earth, m_sun, d_sun - r) - grav_pull(m_earth, m_sun, d_sun)   
dp_C_moon = grav_pull(m_earth, m_moon, d_moon - r) - grav_pull(m_earth, m_moon, d_moon)  

print("Differential pull at C due to the sun : %E g" % dp_C_sun)
print("Differential pull at C due to the moon: %E g" % dp_C_moon)
print("Percentage of the total differential pull associated to the sun : %.1f%%"
    % (dp_C_sun / (dp_C_sun + dp_C_moon) * 100))
print("Percentage of the total differential pull associated to the moon: %.1f%%"
    % (dp_C_moon / (dp_C_sun + dp_C_moon) * 100))

## Location G
## Add your own code here (to verify your answer to the below question Q-6)



In [ ]:
## Run this cell to get question(s)

q = [
    "Q2c_diff_pull_sun",
    "Q2c_diff_pull_moon",
    "Q2c_diff_pull_reflect",
    "Q2c_moon_vs_sun"
]

question_industry.serve(q)

## Part 2: Equilibrium response and spring-neap tide

### Tide-generating forces and equilibrium response

The above computed differential pull of Sun and Moon are both several orders of magnitude smaller than the Earth's own attraction towards its centre, $g$. That means, that the component of the differential pull that is in line with Earth's centre (perpendicular to the Earth's surface) can be neglected, since its magnitude is negligible compared to $g$, and will therefore not make a noticeable difference. The only thing we are left with then, are the *tangential components* of the differential pull. These are the forces that generate the tide by moving the water around the surface of the Earth. This results in the two bulges of water at either side of the Earth, in line with the Sun/Moon. In the equilibrium theory of the tide, the water masses are assumed to cover the entire Earth and respond instantaneously to the tide-generating forces.

### Spring-neap tide cycle 

Depending on the moon phases, the lunar and solar bulges can reinforce each other or dampen each other. If the Sun and Moon are aligned, the forces enhance each other and we get a spring tide (larger tidal range). On the other hand, if they are perpendicular to each other, the lunar and solar differential pull work against each other and we get a neap tide. 

The code below visualizes this concept; it presents a figure that allows you to play around with different constellations of the Moon and Sun, to see what the effect is on the total tidal shape. In the plot, the sun is fixed to the right side of the Earth (see also Figure 3.21 in the book). You can adjust the position of the Moon by setting its angle relative to the position of the sun. So if you set it at 90, the moon will be in a position 90 degrees counterclock-wise from the sun (i.e., they will be perpendicular). In short, this means:

|Angle | Position |
|:-:|-|
| 0   | New moon | 
| 90  | 1st quarter moon | 
| 180 | Full moon |
| 270 | 3rd quarter moon |

By adjusting the figure, we can see how the alignment of the Earth, Moon, and Sun influences the tide experienced by a given point on Earth.  If the tidal bulges align, we get a higher high tide, "spring tide", but if the tidal bulges are out of alignment, we have a reduced high tide, the "neap tide". The same holds for low waters: low tides are lower when everything is aligned (spring tide), and the low tides at neap are not quite as low.

**Note**: the figure only plots the correct total tide for intervals of 90 degrees (0, 90, 180, or 270 degrees)

In [ ]:
# Note: the figure only plots the correct total tide for intervals of 90 degrees (0, 90, 180, or 270 degrees)

angle_moon = 0  # Adjust here the position of the Moon
angle_sun = 0  # Fixed at 0, DON'T change

mod_2c.tide_diagram(angle_sun, angle_moon)

In [ ]:
## Run this cell to get question(s)

q = [
    "Q2c_angle_moon_0",
    "Q2c_angle_moon_180",
    "Q2c_angle_moon_90",
    "Q2c_angle_moon_270"
]

question_industry.serve(q)

## The end 

You have reached the end of this Notebook 2c. You can continue with this week's fourth and last third notebook, Notebook 2d on tidal constituents.